# Exports

This notebooks shows the steps on how we create an export using a project instance.

In [ ]:
from rasterfoundry.api import API
refresh_token = '<your refresh token>'
api = API(refresh_token=refresh_token)

In [ ]:
api.projects

## Create an export directly


### Create a polygon as the mask for the export

In [ ]:
from ipyleaflet import DrawControl

project = api.projects[-1]
m = project.get_map()

dc = DrawControl()
m.add_control(dc)
project.add_to(m)
m

# Draw a polygon in the map below

In [ ]:
def snap_to_360(x_coord):
    """Snap an x coordinate to [-180, 180]
    
    Coordinates coming back from the API for some projects can be
    outside this range, and coordinates in the bbox outside this
    range make the export API upset. When it's upset, it returns
    an array with just a single 0 in it, which is not an accurate
    representation of the project normally.
    """
    return x_coord - round((x_coord + 180) / 360, 0) * 360

def geom_to_bbox(geom):
    coords = geom['geometry']['coordinates'][0]
    min_x = snap_to_360(min([point[0] for point in coords]))
    min_y = min([point[1] for point in coords])
    max_x = snap_to_360(max([point[0] for point in coords]))
    max_y = max([point[1] for point in coords])
    return ','.join(map(str, [min_x, min_y, max_x, max_y]))

bbox = geom_to_bbox(dc.last_draw)


### Create an export through a project instance

In [ ]:
project_export = project.create_export(bbox=bbox, zoom=8, raster_size=1000)

### Wait until the export is finished

In [ ]:
completed_export = project_export.wait_for_completion()
# it will say 'EXPORTED' as the output of this block if the export is finished
completed_export.export_status

In [ ]:
# When the export is done, we will get a list of download URLs
project_export.files

### Download and plot the export

Note: this example requires
[`numpy`](http://www.numpy.org/),
[`matplotlib`](http://matplotlib.org/), and a fairly recent version of
[`rasterio`](https://mapbox.github.io/rasterio/).

If you don't have them, you can run the cell at the bottom of this notebook,
provided your pip installation directory is writable.

In [ ]:
from rasterio.io import MemoryFile
import matplotlib.pyplot as plt

data = project_export.download_file_bytes()

with MemoryFile(data) as memfile:
    with memfile.open() as dataset:
        plt.imshow(dataset.read(1), cmap='viridis')
        
plt.show()

# Installs

In [ ]:
%%bash
pip install rasterio==1.0a12 matplotlib